# JS06 - Query Workers

Query as API (aka Query Workers) enables developers to convert saved C8QL queries into geo-distributed REST APIs quickly. This eliminates the need for separate backend servers & containers for CRUD operations.

## Pre-requisite

Let's assume:
- You have already made a tenant account, and have a username and password
- You have installed the jsC8 drivers as explained in section 01
- You have generated an API Key as explained in section 01


In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## 1. Import Libraries to Workbook

In [ ]:
const jsc8 = require("jsc8");

## 2. Define Login Credentials

In [ ]:
const guest_mail = "email";
const guest_password = "password";

## 3. Define Variables and assign Values for Workbook


In [ ]:
const fed_url = "https://gdn.paas.macrometa.io";
const geo_fabric = "_system";
let collection_name = "person";

## 4. Connect to GDN

Now that you have added your login credentials, lets connect to GDN

In [ ]:
console.log("\n ------- CONNECTION SETUP  ------");
console.log(`tenant: ${guest_mail}, geofabric: ${geo_fabric}`);
// Simple Way
// const client = new jsc8({url: fed_url, token: "XXXX", fabricName: geo_fabric});

// ----- OR -----
// const client = new jsc8({url: fed_url, apiKey: "XXXX", fabricName: geo_fabric});

// To use advanced options
const client = new jsc8(fed_url);
client
  .login(guest_mail, guest_password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));


## 5. Create a Geo Replicated Collection

Before we create the Query Workers lets create a collection that we can work with

In [ ]:
console.log("\n ------- CREATE GEO-REPLICATED COLLECTION  ------");

const createCollection = async () => {
  try {
    const isCollectionExists = await client.hasCollection(collection_name);
    if (isCollectionExists) {
      console.log("Collection exists with name ", collection_name);
      return;
    }
    await client.createKVCollection(collection_name);
    console.log("Collection created! ", collection_name);
  } catch (e) {
    return "Collection creation did not succeed due to " + e;
  }
};

createCollection();

## 6. Create Query Workers

### 6.1. Defining our Queries

In this section we are going to define five Query Worker APIs

In [ ]:
let parameter = { firstname: "", lastname: "", email: "", zipcode: "" };
let insert_data = {
  query: {
    name: "insertRecord",
    value: `INSERT {'firstname':@firstname, 'lastname':@lastname, 'email':@email, 'zipcode':@zipcode, '_key': 'key101'} IN ${collection_name}`,
    parameter: parameter,
  },
};
let get_data = {
  query: {
    name: "getRecords",
    value: `FOR doc IN ${collection_name} RETURN doc`,
  },
};

let update_data = {
  query: {
    name: "updateRecord",
    value: `UPDATE 'key101' WITH { \"lastname\": \"cena\" } IN ${collection_name}`,
  },
};

let delete_data = {
  query: {
    name: "deleteRecord",
    value: `REMOVE 'key101' IN ${collection_name}`,
  },
};

let get_count = {
  query: {
    name: "countRecords",
    value: `RETURN COUNT(FOR doc IN ${collection_name} RETURN 1)`,
  },
};

### 6.2. Creating the Query Worker APIs

The Following Code will create a set of Query Worker APIs in GDN - once complete you can check that they exist from your console access, under Queries, and navigate to "Saved Queries"

In [ ]:
const createRestQL = async () => {
  try {
    console.log("\n ------- CREATE RESTQLs  ------");
    await client.createRestql(
      insert_data.query.name.toString(),
      insert_data.query.value.toString(),
      insert_data.query.parameter
    ); // name: insertRecord

    await client.createRestql(
      get_data.query.name.toString(),
      get_data.query.value.toString(),
      {}
    ); // name: getRecords

    await client.createRestql(
      update_data.query.name.toString(),
      update_data.query.value.toString(),
      {}
    ); // name: updateRecord

    await client.createRestql(
      delete_data.query.name.toString(),
      delete_data.query.value.toString(),
      {}
    ); // name: deleteRecord

    await client.createRestql(
      get_count.query.name.toString(),
      get_count.query.value.toString(),
      {}
    ); // name: countRecords
    console.error("Created restQLs");
  } catch (err) {
    console.error("Error while creating restQLs", err.message);
  }
};

createRestQL();

## 7. Execute Query Workers

**Note: we are going to skip testing the "deleteRecord" API in this section - so that we can demonstrate how to update a Query Worker API first.**

### 7.1. Test the "insertRecord" Query Worker

In [ ]:
const executeRestQL = async () => {
  try {
    console.log("\n ------- EXECUTE RESTQLs ------");
    console.log("\n a. Insert Data");
    let resp = await client.executeRestql(insert_data.query.name.toString(), {
      firstname: "john",
      lastname: "doe",
      email: "john.doe@macrometa.io",
      zipcode: "511037",
    });
    console.log(resp.result);

    console.log("\n b. Get Data");
    resp = await client.executeRestql(get_data.query.name.toString(), {});
    console.log(resp.result);
  } catch (err) {
    console.error("Error while executing restQLs", err.message);
  }
};

executeRestQL();

### 7.2. Test the "updateRecord" Query Worker

In [ ]:
const updateRestQL = async () => {
  try {
    console.log("\n ------- EXECUTE RESTQLs ------");

    console.log("\n c. Update Data");
    resp = await client.executeRestql(update_data.query.name.toString(), {});
    console.log(resp.result);

    console.log("\n d. Get Data");
    resp = await client.executeRestql(get_data.query.name.toString(), {});
    console.log(resp.result);
  } catch (err) {
    console.error("Error while executing restQL", err.message);
  }
};

updateRestQL();

### 7.3. Test the "getRecords" Query Worker

In [ ]:
const getRecordsRestQL = async () => {
    try {
  console.log("\n ------- EXECUTE RESTQLs ------");

  console.log("\n d. Get Records");
  resp = await client.executeRestql(get_data.query.name.toString(), {});
  console.log(resp.result);
     }catch(err) {
        console.error("Error while get restQL", err.message)
    }
};

getRecordsRestQL();

### 7.4. Test the "countRecords" Query Worker

In [ ]:
const countRecordsRestQL = async () => {
  try {
    console.log("\n ------- EXECUTE RESTQLs ------");

    console.log("\n e. Count Records");
    resp = await client.executeRestql(get_count.query.name.toString(), {});
    console.log(resp.result);
  } catch (err) {
    console.error("Error while executing restQL", err.message);
  }
};

countRecordsRestQL();

## 8. Updating a Query Worker API

### 8.1. Test that we can query GDN and see the document

A common requirement when building an application is to make a change to an API and update it. 

To demonstrate this, lets first query the database using the "execute_query" method we learnt in an earlier section

Once run, we should see the document in the cell output in JSON format so we can read it easily.

In [ ]:
console.log('Get docs in "person" collection');

const c8Queries = async () => {
  try {
    let docs = await client.executeQuery(
      `FOR doc IN ${collection_name} RETURN doc`
    );
    console.log(docs);
  } catch (err) {
    console.error("Error while fetching records", err.message);
  }
};

c8Queries();

### 8.2. Updating a Query Worker API

The follow cell show us how to add a new parameter to the Query, in this example we are going to add a the parameter "phone" to the query.

In [ ]:
value = `INSERT {'firstname':@firstname, 'lastname':@lastname, 'email':@email, 'zipcode':@zipcode, 'phone': @phone} IN ${collection_name}`;

insert_data = {
  query: { name: "insertRecord", parameter: parameter, value: value },
};

const updateRestql = async () => {
  try {
    console.log("\n ------- UPDATE RESTQLs  ------");
    response = await client.updateRestql(
      insert_data.query.name,
      insert_data.query.value,
      insert_data.query.parameter
    );
    console.log(response);
  } catch (err) {
    console.error("Error while updating query worker", err.message);
  }
};

updateRestql();

### 8.3. Testing the updated Query Worker API

Now that we have updated the Query, lets add a new record using it to test that it works.

In [ ]:
const executeUpdateRestql = async () => {
  try {
    console.log("\n ------- EXECUTE RESTQLs ------");
    console.log("\n Insert data with updated query worker");
    let resp = await client.executeRestql(insert_data.query.name, {
      firstname: "john",
      lastname: "doe",
      email: "john.doe@macrometa.io",
      zipcode: "511037",
      phone: "213-555-9578",
    });
    console.log(resp.result);
  } catch (err) {
    console.error("Error while execute RestQL", err.message);
  }
};

executeUpdateRestql();

### 8.4. Checking the data in the collection

Now lets check that the data has been updated in the record by using the "execute_query" method - we should see two documents. 

1. the original document without a phone number, and
2. a second document, but now with a phone number added

Hint: If you would like to return to the step “countRecords” and re run the code, you should see the count value increase by 1

In [ ]:
console.log(
  'Get docs in "person" collection, after executing the updated query'
);

const executeQueries = async () => {
  try {
    let docs = await client.executeQuery(
      `FOR doc IN ${collection_name} RETURN doc`
    );
    console.log(docs);
  } catch (err) {
    console.error("Error while fetching records", err.message);
  }
};

executeQueries();

## 8.5. Deleting a record using a Query Worker API

We might want to delete our Query Worker APIs completely.

For each Query Worker we want to remove we use the “delete_restql” method as shown in the cell below

In [ ]:
const executeDeleteRestQL = async () => {
  try {
    console.log("Delete data....");
    let resp = await client.executeRestql("deleteRecord");
    console.log(resp.result);
  } catch (err) {
    console.error("Error while execute restQL", err.message);
  }
};

executeDeleteRestQL();

## 9. Delete Query Workers

We might want to delete our Query Worker APIs completely.

For each Query Worker we want to remove we use the "deleteRestql" method as shown in the cell below

In [ ]:
const deleteRestQL = async () => {
  try {
    console.log("\n ------- DELETE RESTQLs ------");
    await client.deleteRestql(insert_data.query.name.toString());
    await client.deleteRestql(get_data.query.name.toString());
    await client.deleteRestql(update_data.query.name.toString());
    await client.deleteRestql(get_data.query.name.toString());
    await client.deleteRestql(get_count.query.name.toString());
    await client.deleteRestql(delete_data.query.name.toString());
    console.log("\n ------- DONE  ------");
  } catch (err) {
    console.error("Error while delete restQL", err.message);
  }
};

deleteRestQL();

## 10. Delete the Collection

Now let us remove the collection we added during this Tutorial

In [ ]:
const deleteCollection = async () => {
  try {
    await client.deleteCollection(collection_name);
    console.log("Collection Deleted: ", collection_name);
  } catch (err) {
    console.log("Error while deleting collection ", err.message);
  }
};

deleteCollection();

## Section Completed!

Congratulations!, another tutorial completed.

**Note: To use the created Query Worker APIs you should use the API reference documentation for detailed instructions.**

e.g. /_fabric/{fabric}/_api/restql/{name} 

where {name} is the name of your Query Worker API e.g. "insertRecord"

